# Data Merge - Team 8 Departments

This notebook merges all raw data files for Team 8 department forecasting.

**Team 8 Departments:**
- Dept 6: Cameras and Supplies
- Dept 9: Sporting Goods
- Dept 41: Team Sports
- Dept 67: Celebration
- Dept 90: Dairy

**Goal:** Create merged datasets ready for modeling

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Raw Data

In [2]:
# Load all datasets
print("Loading raw data...\n")

# 1. Cases data
cases = pd.read_csv('inbound_cases_team8.csv')
cases['dt'] = pd.to_datetime(cases['dt'])
print(f"✓ Cases: {cases.shape}")

# 2. Trucks data
trucks = pd.read_csv('trucks.csv')
trucks['dt'] = pd.to_datetime(trucks['dt'])
print(f"✓ Trucks: {trucks.shape}")

# 3. Stores data
stores = pd.read_excel('stores_data.xlsx')
stores = stores.rename(columns={'store_id5': 'store_id'})
print(f"✓ Stores: {stores.shape}")

# 4. Department mapping
dept_mapping = pd.read_excel('student mapping share.xlsx')
print(f"✓ Department mapping: {dept_mapping.shape}")

Loading raw data...

✓ Cases: (275000, 5)
✓ Trucks: (52200, 3)
✓ Stores: (100, 4)
✓ Department mapping: (57, 9)


## 2. Inspect Data

In [3]:
print("=" * 80)
print("CASES DATA")
print("=" * 80)
print(f"\nShape: {cases.shape}")
print(f"\nColumns: {list(cases.columns)}")
print(f"\nDate range: {cases['dt'].min().date()} to {cases['dt'].max().date()}")
print(f"Missing cases: {cases['cases'].isna().sum():,}")
print(f"\nFirst few rows:")
display(cases.head())
print(f"\nData types:")
print(cases.dtypes)

CASES DATA

Shape: (275000, 5)

Columns: ['dept_id', 'store_id', 'dt', 'cases', 'student_group']

Date range: 2024-03-14 to 2025-09-14
Missing cases: 14,000

First few rows:


,dept_id,store_id,dt,cases,student_group
0,6,10002,2025-02-20,63.0,8
1,6,10002,2025-02-28,56.0,8
2,6,10002,2025-02-19,62.0,8
3,90,10001,2025-02-25,67.0,8
4,9,10004,2025-02-07,53.0,8



Data types:
dept_id                   int64
store_id                  int64
dt               datetime64[ns]
cases                   float64
student_group             int64
dtype: object


In [4]:
print("=" * 80)
print("TRUCKS DATA")
print("=" * 80)
print(f"\nShape: {trucks.shape}")
print(f"\nColumns: {list(trucks.columns)}")
print(f"\nDate range: {trucks['dt'].min().date()} to {trucks['dt'].max().date()}")
print(f"Missing trucks: {trucks['trucks'].isna().sum()}")
print(f"\nFirst few rows:")
display(trucks.head())
print(f"\nTrucks distribution:")
print(trucks['trucks'].value_counts().sort_index())

TRUCKS DATA

Shape: (52200, 3)

Columns: ['store_id', 'dt', 'trucks']

Date range: 2024-03-14 to 2025-08-17
Missing trucks: 0

First few rows:


,store_id,dt,trucks
0,10036,2025-02-04,4
1,10085,2025-02-22,4
2,10001,2024-12-11,4
3,10001,2024-12-19,4
4,10047,2024-12-22,4



Trucks distribution:
trucks
1       20
2    43312
3     8836
4       32
Name: count, dtype: int64


In [5]:
print("=" * 80)
print("STORES DATA")
print("=" * 80)
print(f"\nShape: {stores.shape}")
print(f"\nColumns: {list(stores.columns)}")
print(f"Missing values: {stores.isnull().sum().sum()}")
print(f"\nFirst few rows:")
display(stores.head())
print(f"\nStates distribution:")
print(stores['state_name'].value_counts().head(10))

STORES DATA

Shape: (100, 4)

Columns: ['store_id', 'state_name', 'market_area_nbr', 'region_nbr']
Missing values: 0

First few rows:


,store_id,state_name,market_area_nbr,region_nbr
0,10057,AL,1,14
1,10042,AL,7,14
2,10064,GA,23,2
3,10073,SC,36,4
4,10013,SC,36,4



States distribution:
state_name
TX    15
FL     9
AR     6
KY     4
IN     4
VA     4
GA     4
LA     4
SC     4
CA     3
Name: count, dtype: int64


In [6]:
print("=" * 80)
print("DEPARTMENT MAPPING")
print("=" * 80)
print(f"\nShape: {dept_mapping.shape}")
print(f"\nColumns: {list(dept_mapping.columns)}")
print(f"\nAll columns:")
display(dept_mapping.head())

# Clean up department mapping - keep only relevant columns
dept_mapping_clean = dept_mapping[['dept_id', 'dept_desc', 'gmm_name', 'dmm_name', 'student_group']].copy()
dept_mapping_clean = dept_mapping_clean.dropna(subset=['dept_id', 'student_group'])

print(f"\nCleaned department mapping shape: {dept_mapping_clean.shape}")

DEPARTMENT MAPPING

Shape: (57, 9)

Columns: ['dept_id', 'dept_desc', 'gmm_name', 'dmm_name', 'student_group', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']

All columns:


,dept_id,dept_desc,gmm_name,dmm_name,student_group,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,38,PHARMACY RX,HEALTH AND WELLNESS,HEALTH AND WELLNESS,1,NaN,NaN,NaN,NaN
1,92,GROCERY DRY GOODS,FOOD,PANTRY,1,NaN,NaN,NaN,NaN
2,81,COMM BREAD,FOOD,PANTRY,1,NaN,NaN,NaN,NaN
3,23,MENS,GENERAL MERCHANDISE,APPAREL,2,NaN,NaN,NaN,NaN
4,96,"LIQUOR,WINE,BEER",FOOD,CHILLED ALCOHOL AND CONVENIENCE,2,NaN,NaN,NaN,NaN



Cleaned department mapping shape: (57, 5)


## 3. Filter Team 8 Departments

In [7]:
# Get Team 8 departments
team8_depts = dept_mapping_clean[dept_mapping_clean['student_group'] == 8].copy()

print("=" * 80)
print("TEAM 8 DEPARTMENTS")
print("=" * 80)
print()
display(team8_depts)

team8_dept_ids = team8_depts['dept_id'].tolist()
print(f"\nTeam 8 Department IDs: {team8_dept_ids}")

TEAM 8 DEPARTMENTS



,dept_id,dept_desc,gmm_name,dmm_name,student_group
28,41,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL,8
29,90,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE,8
30,67,CELEBRATION,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL,8
31,6,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL,8
32,9,SPORTING GOODS,GENERAL MERCHANDISE,HARDLINES,8



Team 8 Department IDs: [41, 90, 67, 6, 9]


In [8]:
# Verify cases data already filtered to Team 8
print("Verifying cases data...")
print(f"Unique departments in cases: {sorted(cases['dept_id'].unique())}")
print(f"Team 8 departments: {sorted(team8_dept_ids)}")
print(f"\nMatch: {set(cases['dept_id'].unique()) == set(team8_dept_ids)}")

if set(cases['dept_id'].unique()) == set(team8_dept_ids):
    print("\n✓ Cases data is already filtered to Team 8 departments only!")
else:
    print("\n⚠ Warning: Cases data contains other departments!")

Verifying cases data...
Unique departments in cases: [6, 9, 41, 67, 90]
Team 8 departments: [6, 9, 41, 67, 90]

Match: True

✓ Cases data is already filtered to Team 8 departments only!


## 4. Merge All Data

**Merge Strategy:**
1. Start with cases data (store × dept × day)
2. Add trucks data (store × day) - LEFT JOIN
3. Add stores data (store) - LEFT JOIN
4. Add department info (dept) - LEFT JOIN

In [9]:
print("Merging datasets...\n")

# Start with cases
merged = cases.copy()
print(f"Starting with cases: {merged.shape}")

# Merge with trucks (on store_id + dt)
merged = merged.merge(trucks, on=['store_id', 'dt'], how='left')
print(f"After merging trucks: {merged.shape}")

# Merge with stores (on store_id)
merged = merged.merge(stores, on='store_id', how='left')
print(f"After merging stores: {merged.shape}")

# Merge with department info (on dept_id)
dept_info = team8_depts[['dept_id', 'dept_desc', 'gmm_name', 'dmm_name']].copy()
merged = merged.merge(dept_info, on='dept_id', how='left')
print(f"After merging dept info: {merged.shape}")

print(f"\n✓ All data merged successfully!")
print(f"Final shape: {merged.shape}")

Merging datasets...

Starting with cases: (275000, 5)
After merging trucks: (275000, 6)
After merging stores: (275000, 9)
After merging dept info: (275000, 12)

✓ All data merged successfully!
Final shape: (275000, 12)


## 5. Data Quality Checks

In [10]:
print("=" * 80)
print("DATA QUALITY CHECKS")
print("=" * 80)

print("\n1. MISSING VALUES")
print("=" * 40)
missing = merged.isnull().sum()
missing_pct = (missing / len(merged) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percent': missing_pct
}).sort_values('Missing_Count', ascending=False)
print(missing_df[missing_df['Missing_Count'] > 0])

print("\n2. DATE RANGE")
print("=" * 40)
print(f"Start: {merged['dt'].min().date()}")
print(f"End: {merged['dt'].max().date()}")
print(f"Days: {merged['dt'].nunique()}")

print("\n3. GRANULARITY")
print("=" * 40)
print(f"Stores: {merged['store_id'].nunique()}")
print(f"Departments: {merged['dept_id'].nunique()}")
print(f"Store-Dept combinations: {merged.groupby(['store_id', 'dept_id']).ngroups}")

print("\n4. FORECAST PERIOD")
print("=" * 40)
forecast_data = merged[merged['cases'].isna()]
historical_data = merged[merged['cases'].notna()]
print(f"Historical records (with cases): {len(historical_data):,}")
print(f"Forecast records (missing cases): {len(forecast_data):,}")
print(f"Forecast date range: {forecast_data['dt'].min().date()} to {forecast_data['dt'].max().date()}")
print(f"Forecast days: {forecast_data['dt'].nunique()}")

DATA QUALITY CHECKS

1. MISSING VALUES
        Missing_Count  Missing_Percent
cases           14000             5.09
trucks          14000             5.09

2. DATE RANGE
Start: 2024-03-14
End: 2025-09-14
Days: 550

3. GRANULARITY
Stores: 100
Departments: 5
Store-Dept combinations: 500

4. FORECAST PERIOD
Historical records (with cases): 261,000
Forecast records (missing cases): 14,000
Forecast date range: 2025-08-18 to 2025-09-14
Forecast days: 28


In [11]:
print("=" * 80)
print("MERGED DATA SAMPLE")
print("=" * 80)

print("\nColumns:")
print(merged.columns.tolist())

print("\nFirst 10 rows:")
display(merged.head(10))

print("\nData types:")
print(merged.dtypes)

MERGED DATA SAMPLE

Columns:
['dept_id', 'store_id', 'dt', 'cases', 'student_group', 'trucks', 'state_name', 'market_area_nbr', 'region_nbr', 'dept_desc', 'gmm_name', 'dmm_name']

First 10 rows:


,dept_id,store_id,dt,cases,student_group,trucks,state_name,market_area_nbr,region_nbr,dept_desc,gmm_name,dmm_name
0,6,10002,2025-02-20,63.0,8,2.0,NC,296,26,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL
1,6,10002,2025-02-28,56.0,8,2.0,NC,296,26,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL
2,6,10002,2025-02-19,62.0,8,2.0,NC,296,26,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL
3,90,10001,2025-02-25,67.0,8,3.0,MD,285,22,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE
4,9,10004,2025-02-07,53.0,8,2.0,LA,66,13,SPORTING GOODS,GENERAL MERCHANDISE,HARDLINES
5,41,10004,2025-02-15,46.0,8,2.0,LA,66,13,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
6,41,10004,2025-02-14,45.0,8,2.0,LA,66,13,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
7,41,10001,2025-02-19,70.0,8,3.0,MD,285,22,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
8,67,10004,2025-02-05,63.0,8,2.0,LA,66,13,CELEBRATION,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL
9,67,10001,2025-02-02,66.0,8,2.0,MD,285,22,CELEBRATION,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL



Data types:
dept_id                     int64
store_id                    int64
dt                 datetime64[ns]
cases                     float64
student_group               int64
trucks                    float64
state_name                 object
market_area_nbr             int64
region_nbr                  int64
dept_desc                  object
gmm_name                   object
dmm_name                   object
dtype: object


In [12]:
# Show sample from forecast period
print("=" * 80)
print("FORECAST PERIOD SAMPLE (Missing Cases)")
print("=" * 80)
display(merged[merged['cases'].isna()].head(10))

FORECAST PERIOD SAMPLE (Missing Cases)


,dept_id,store_id,dt,cases,student_group,trucks,state_name,market_area_nbr,region_nbr,dept_desc,gmm_name,dmm_name
261000,41,10097,2025-09-14,NaN,8,NaN,KY,498,59,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
261001,90,10098,2025-09-04,NaN,8,NaN,AR,327,42,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE
261002,90,10097,2025-09-06,NaN,8,NaN,KY,498,59,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE
261003,41,10097,2025-09-01,NaN,8,NaN,KY,498,59,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
261004,90,10097,2025-09-02,NaN,8,NaN,KY,498,59,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE
261005,6,10099,2025-09-10,NaN,8,NaN,VA,287,75,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL
261006,41,10098,2025-09-01,NaN,8,NaN,AR,327,42,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL
261007,9,10096,2025-09-12,NaN,8,NaN,IL,262,25,SPORTING GOODS,GENERAL MERCHANDISE,HARDLINES
261008,9,10096,2025-09-11,NaN,8,NaN,IL,262,25,SPORTING GOODS,GENERAL MERCHANDISE,HARDLINES
261009,67,10098,2025-09-04,NaN,8,NaN,AR,327,42,CELEBRATION,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL


## 6. Create Reference Tables

Create smaller lookup tables for stores and departments

In [13]:
# Stores reference table
stores_ref = stores.copy()
print(f"Stores reference table: {stores_ref.shape}")
display(stores_ref.head())

# Department reference table (Team 8 only)
dept_ref = team8_depts.copy()
print(f"\nDepartment reference table (Team 8): {dept_ref.shape}")
display(dept_ref)

Stores reference table: (100, 4)


,store_id,state_name,market_area_nbr,region_nbr
0,10057,AL,1,14
1,10042,AL,7,14
2,10064,GA,23,2
3,10073,SC,36,4
4,10013,SC,36,4



Department reference table (Team 8): (5, 5)


,dept_id,dept_desc,gmm_name,dmm_name,student_group
28,41,TEAM SPORTS,GENERAL MERCHANDISE,APPAREL,8
29,90,DAIRY,FOOD,CHILLED ALCOHOL AND CONVENIENCE,8
30,67,CELEBRATION,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL,8
31,6,CAMERAS AND SUPPLIES,GENERAL MERCHANDISE,ENTERTAINMENT TOYS AND SEASONAL,8
32,9,SPORTING GOODS,GENERAL MERCHANDISE,HARDLINES,8


## 7. Save Merged Data

In [14]:
import os

# Create directories if they don't exist
os.makedirs('data', exist_ok=True)
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

print("Saving processed data...\n")

# Save merged data
merged.to_csv('data/processed/merged_data.csv', index=False)
print(f"✓ Saved: data/processed/merged_data.csv ({merged.shape})")

# Save reference tables
stores_ref.to_csv('data/processed/stores_reference.csv', index=False)
print(f"✓ Saved: data/processed/stores_reference.csv ({stores_ref.shape})")

dept_ref.to_csv('data/processed/dept_reference.csv', index=False)
print(f"✓ Saved: data/processed/dept_reference.csv ({dept_ref.shape})")

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"\n✓ Merged dataset: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
print(f"✓ Team 8 departments: {merged['dept_id'].nunique()} departments")
print(f"✓ Stores: {merged['store_id'].nunique()} stores")
print(f"✓ Date range: {merged['dt'].min().date()} to {merged['dt'].max().date()}")
print(f"✓ Historical data: {len(historical_data):,} records")
print(f"✓ Forecast needed: {len(forecast_data):,} predictions")
print(f"\nAll files saved to data/processed/")

Saving processed data...

✓ Saved: data/processed/merged_data.csv ((275000, 12))
✓ Saved: data/processed/stores_reference.csv ((100, 4))
✓ Saved: data/processed/dept_reference.csv ((5, 5))

SUMMARY

✓ Merged dataset: 275,000 rows × 12 columns
✓ Team 8 departments: 5 departments
✓ Stores: 100 stores
✓ Date range: 2024-03-14 to 2025-09-14
✓ Historical data: 261,000 records
✓ Forecast needed: 14,000 predictions

All files saved to data/processed/


## 8. Final Column Overview

In [15]:
print("=" * 80)
print("FINAL MERGED DATA COLUMNS")
print("=" * 80)

column_info = pd.DataFrame({
    'Column': merged.columns,
    'Type': merged.dtypes.values,
    'Non-Null': merged.count().values,
    'Null': merged.isnull().sum().values,
    'Null %': (merged.isnull().sum() / len(merged) * 100).round(2).values
})

display(column_info)

print("\n" + "=" * 80)
print("READY FOR FEATURE ENGINEERING & MODELING!")
print("=" * 80)

FINAL MERGED DATA COLUMNS


,Column,Type,Non-Null,Null,Null %
0,dept_id,int64,275000,0,0.00
1,store_id,int64,275000,0,0.00
2,dt,datetime64[ns],275000,0,0.00
3,cases,float64,261000,14000,5.09
4,student_group,int64,275000,0,0.00
5,trucks,float64,261000,14000,5.09
6,state_name,object,275000,0,0.00
7,market_area_nbr,int64,275000,0,0.00
8,region_nbr,int64,275000,0,0.00
9,dept_desc,object,275000,0,0.00



READY FOR FEATURE ENGINEERING & MODELING!
